In [8]:
import math
import random

class Value:
    
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None #backpropagation, by default nothing to do
        self._prev = set(_children) #record the children
        self._op = _op
        self.label = label
    
    #repr makes the printed values readable
    def __repr__(self):
        return f"Value(data={self.data})"
    
    
    #use double score in phyton to change default operators
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other) #this enables inline operation on Value like a+1
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            # = instead of += below in original implementation
            self.grad += 1.0 * out.grad  #transfer grad + bugfix accumulate due to multivariable case of chain rule
            other.grad += 1.0 * out.grad 
        out._backward = _backward
        
        return out
    
    def __radd__(self, other): #this is for reserve order mult like 2+a, since above only works a.__add__(2) but not in reserve
        return self * other
    
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other) #this enables inline operation on Value like a*2
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            # = instead of += below in original implementation
            self.grad += other.data * out.grad #chainrule + bugfix: accumulate due to multivariable case of chain rule
            other.grad += self.data * out.grad 
        out._backward = _backward
        
        return out
    
    def __rmul__(self, other): #this is for reserve order mult like 2*a, since above only works a.__mul__(2) but not in reserve
        return self * other
    
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers"
        out = Value(self.data**other, (self,), f'**{other}')
        
        def _backward():
            self.grad += other * (self.data**(other-1)) * out.grad #derivate of x**n is n * x**(n-1)
        out._backward = _backward
        
        return out
    
    
    def __truediv__(self, other):
        return self * other**-1
    
    
    def __neg__(self):
        return self * -1
    
    def __sub__(self, other):
        return self + (-other)
    
                    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')
        
        def _backward():
            # = instead of += below in original implementation
            self.grad += (1 - t**2) * out.grad #local derivate of tanh times out.grad due to chain rule + bugfix: accumulate due to multivariable case of chain rule
        out._backward = _backward
                
        return out
    
                    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self ,), 'exp')
        
        def _backward():
            self.grad += out.data * out.grad # because d(e**x)/dx = e**x * ln_e = e**x since ln_e=1
        out._backward = _backward
        
        return out
                    
    
    def backward(self):

        topo = []
        visited = set()
        def build_topo(v): #build topo from the top object down to its all children, start adding the bottom most child
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        self.grad = 1.0 #required because its initilized as 0, but is 1.0 for topmost node
        for node in reversed(topo): #iterating in topo in reserve order, since top node shall the start point for backpropagation
            node._backward()
            
class Neuron:
    
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)] #weights
        self.b = Value(random.uniform(-1,1)) #bias
        
    def __call__(self, x):
        # w * x +b
        # print(list(zip(self.w, x))) # iteration over tuples
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b) #activation
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]

class Layer:
    
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    
    def __call__(self,x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs #return single element if only one
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
        #longer form of above one liner
        #params = []
        #for neuron in self.neurons:
        #    ps = neuron.parameters()
        #    params.extend(ps)
        #return params
        
class MLP: #Multi Layer Perceptrons
    
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i],sz[i+1]) for i in range(len(nouts))]
        
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [32]:
# reinit input and the neural net
xs = [[1.1], [1.5], [3.0], [6.0]] #inputs
ys = [0.25, 0.40, 0.75, 1.0] #desired targets
n = MLP(1, [4,4,1]) #neural net

# putting back and forward passes together in a training loop
# this is called a gradient descent
for k in range(100000):

    #forward pass
    ydred = [n(x) for x in xs]
    loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ydred)])
    
    #zerograd
    for p in n.parameters():
        p.grad = 0.0
    
    #backward pass
    loss.backward()
    
    #update
    for p in n.parameters():
        p.data += -0.05 * p.grad
        
    print(k, loss.data)
    
    if loss.data < 10e-3:
        break    # break here when sufficient precision is achieved

# print results
for i in range(0, len(ydred)):
    print("Target :", ys[i], " | Guess :", ydred[i].data)
print("Solved with Neural Network with", len(n.parameters()), "no. parameters with loss", loss.data)

0 0.2974447086810337
1 0.2544127965070754
2 0.24337035213709007
3 0.23345580094146895
4 0.22363195874042338
5 0.2138050455783443
6 0.20404987502336297
7 0.1944800992735968
8 0.1852079545955001
9 0.1763251695460982
10 0.16789308279536297
11 0.15993979674990777
12 0.15246324738119185
13 0.145438289571634
14 0.1388253538001043
15 0.13257850835274124
16 0.1266516332559436
17 0.12100235994346975
18 0.1155940785606103
19 0.11039657642718365
20 0.10538585600368952
21 0.10054354226275125
22 0.09585613634758436
23 0.09131425347588568
24 0.08691190685384104
25 0.08264585733999552
26 0.07851502890188791
27 0.07451998297500269
28 0.07066244436277189
29 0.06694487359464475
30 0.06337008364176558
31 0.05994090150550646
32 0.05665987692433154
33 0.05352904109791225
34 0.05054971792681508
35 0.047722388985216024
36 0.04504661155945881
37 0.04252098692708631
38 0.04014317395137293
39 0.037909941311065246
40 0.03581725047618203
41 0.03386036097792504
42 0.03203394960179459
43 0.030332235771652644
44 0.0

In [33]:
n([2.0])

Value(data=0.5531896434556004)